# Agregaciones y Agrupaciones con Pandas

Este notebook demuestra cómo resumir y agrupar datos usando agregaciones, groupby y pivot tables.

**Referencia:** [Agregaciones y Agrupaciones](../pandas/python-para-datos/04-agregaciones.md)

**Objetivos:**
- Realizar agregaciones básicas (sum, mean, min, max, etc.)
- Agrupar datos con groupby()
- Usar múltiples funciones de agregación
- Crear tablas pivot para visualizar datos
- Encontrar top N por categoría

## 1. Importar librerías y cargar datos

In [ ]:
import pandas as pd
import numpy as np

# Cargar datos desde el CSV de ejemplo
df = pd.read_csv('../data/ventas.csv')

# Convertir fecha a datetime para trabajar con meses
df['fecha'] = pd.to_datetime(df['fecha'])
df['mes'] = df['fecha'].dt.to_period('M')

print(f"✅ Datos cargados: {len(df)} registros")
print(f"\nColumnas: {df.columns.tolist()}")
print("\nPrimeras filas:")
df.head()

## 2. Agregaciones básicas

In [ ]:
# Agregaciones simples en una columna numérica
print("=== AGREGACIONES BÁSICAS ===")
print(f"Total de ventas: €{df['total'].sum():,.2f}")
print(f"Precio promedio: €{df['precio'].mean():,.2f}")
print(f"Precio mediano: €{df['precio'].median():,.2f}")
print(f"Desviación estándar: €{df['precio'].std():,.2f}")
print(f"Precio mínimo: €{df['precio'].min():,.2f}")
print(f"Precio máximo: €{df['precio'].max():,.2f}")
print(f"Total de registros: {df['id'].count()}")
print(f"Cantidad total vendida: {df['cantidad'].sum()} unidades")

# Todas las agregaciones numéricas automáticas
print("\n=== ESTADÍSTICAS DESCRIPTIVAS (describe) ===")
print(df.describe())

# Agregaciones personalizadas con agg()
print("\n=== AGREGACIONES PERSONALIZADAS ===")
agregaciones = df.agg({
    'precio': ['mean', 'min', 'max', 'std'],
    'total': ['sum', 'mean'],
    'cantidad': 'sum'
})
print(agregaciones)

## 3. Agrupaciones básicas con groupby()

In [ ]:
# Agrupar por ciudad y calcular promedio de precio
print("=== PRECIO PROMEDIO POR CIUDAD ===")
precio_por_ciudad = df.groupby('ciudad')['precio'].mean()
print(precio_por_ciudad)

# Agrupar por categoría y calcular total de ventas
print("\n=== TOTAL DE VENTAS POR CATEGORÍA ===")
total_por_categoria = df.groupby('categoria')['total'].sum()
print(total_por_categoria)

# Múltiples columnas: ciudad y categoría
print("\n=== PRECIO PROMEDIO POR CIUDAD Y CATEGORÍA ===")
precio_ciudad_cat = df.groupby(['ciudad', 'categoria'])['precio'].mean()
print(precio_ciudad_cat)

## 4. Múltiples funciones de agregación

In [ ]:
# Diferentes funciones por columna
print("=== ESTADÍSTICAS POR CIUDAD ===")
estadisticas_ciudad = df.groupby('ciudad').agg({
    'precio': ['mean', 'min', 'max'],
    'total': 'sum',
    'id': 'count'  # Número de ventas
})
print(estadisticas_ciudad)

# Estadísticas por categoría
print("\n=== ESTADÍSTICAS POR CATEGORÍA ===")
estadisticas_categoria = df.groupby('categoria').agg({
    'precio': ['mean', 'min', 'max', 'std'],
    'total': ['sum', 'mean'],
    'cantidad': 'sum'
})
print(estadisticas_categoria)

## 5. Named aggregations (más legible)

In [ ]:
# Con nombres personalizados (más legible)
print("=== RESUMEN POR CIUDAD (NAMED AGGREGATIONS) ===")
resumen_ciudad = df.groupby('ciudad').agg(
    precio_promedio=('precio', 'mean'),
    total_ventas=('total', 'sum'),
    num_transacciones=('id', 'count'),
    cantidad_total=('cantidad', 'sum')
)
print(resumen_ciudad)

# Resumen por categoría
print("\n=== RESUMEN POR CATEGORÍA ===")
resumen_categoria = df.groupby('categoria').agg(
    precio_promedio=('precio', 'mean'),
    precio_maximo=('precio', 'max'),
    total_ingresos=('total', 'sum'),
    productos_unicos=('producto', 'nunique')
)
print(resumen_categoria)

## 6. Pivot tables

In [ ]:
# Crear tabla pivot: precio promedio por ciudad y categoría
print("=== PRECIO PROMEDIO: CIUDAD (filas) × CATEGORÍA (columnas) ===")
pivot_precio = df.pivot_table(
    values='precio',
    index='ciudad',
    columns='categoria',
    aggfunc='mean'
)
print(pivot_precio)

# Tabla pivot con total de ventas
print("\n=== TOTAL DE VENTAS: CIUDAD × CATEGORÍA ===")
pivot_total = df.pivot_table(
    values='total',
    index='ciudad',
    columns='categoria',
    aggfunc='sum',
    fill_value=0  # Rellenar con 0 si no hay datos
)
print(pivot_total)

In [ ]:
# Tabla pivot con múltiples funciones
print("=== MÚLTIPLES FUNCIONES: CIUDAD × CATEGORÍA ===")
pivot_multi = df.pivot_table(
    values='total',
    index='ciudad',
    columns='categoria',
    aggfunc=['sum', 'mean', 'count'],
    fill_value=0
)
print(pivot_multi)

# Tabla pivot con fechas (agrupar por mes)
print("\n=== VENTAS POR MES Y CATEGORÍA ===")
pivot_mes = df.pivot_table(
    values='total',
    index='mes',
    columns='categoria',
    aggfunc='sum',
    fill_value=0
)
print(pivot_mes)

## 7. Top N por categoría

In [ ]:
# Top 3 productos más vendidos por categoría (usando cantidad)
print("=== TOP 3 PRODUCTOS MÁS VENDIDOS POR CATEGORÍA ===")
top_productos = df.groupby(['categoria', 'producto'])['cantidad'].sum().reset_index()
top_productos = top_productos.sort_values(['categoria', 'cantidad'], ascending=[True, False])
top_3_por_categoria = top_productos.groupby('categoria').head(3)
print(top_3_por_categoria)

# Top 3 ciudades con mayores ingresos
print("\n=== TOP 3 CIUDADES CON MAYORES INGRESOS ===")
top_ciudades = df.groupby('ciudad')['total'].sum().sort_values(ascending=False).head(3)
print(top_ciudades)

# Mes con mayores ventas
print("\n=== MES CON MAYORES VENTAS ===")
ventas_por_mes = df.groupby('mes')['total'].sum().sort_values(ascending=False)
print(ventas_por_mes)
print(f"\nMejor mes: {ventas_por_mes.index[0]} con €{ventas_por_mes.iloc[0]:,.2f}")

## 8. Resumen completo: Análisis de ventas

Este ejemplo combina todas las técnicas aprendidas para crear un análisis completo.

In [ ]:
print("=" * 60)
print("ANÁLISIS COMPLETO DE VENTAS")
print("=" * 60)

# 1. Métricas generales
print("\n📊 MÉTRICAS GENERALES:")
print(f"Total de ingresos: €{df['total'].sum():,.2f}")
print(f"Promedio por transacción: €{df['total'].mean():,.2f}")
print(f"Total de transacciones: {len(df)}")
print(f"Productos únicos: {df['producto'].nunique()}")

# 2. Por categoría
print("\n📦 POR CATEGORÍA:")
resumen_cat = df.groupby('categoria').agg(
    ingresos=('total', 'sum'),
    transacciones=('id', 'count'),
    precio_promedio=('precio', 'mean')
).sort_values('ingresos', ascending=False)
print(resumen_cat)

# 3. Por ciudad
print("\n🏙️ POR CIUDAD:")
resumen_ciudad = df.groupby('ciudad').agg(
    ingresos=('total', 'sum'),
    transacciones=('id', 'count')
).sort_values('ingresos', ascending=False)
print(resumen_ciudad)

# 4. Por mes
print("\n📅 POR MES:")
resumen_mes = df.groupby('mes').agg(
    ingresos=('total', 'sum'),
    transacciones=('id', 'count')
).sort_values('ingresos', ascending=False)
print(resumen_mes)

print("\n" + "=" * 60)